In [3]:
import pandas as pd
import sqlite3
import os

dir = os.getcwd()
data_dir = os.path.join(dir, "Data")

## Loading the Data
The data is stored in two excel sheets. One sheet, Leetcode Problems, contain the problem name, link, unique identifying problem number (primary key), and difficulty. Another sheet, Leetcode Top 150 Solutions, contains the solution with the most votes, length of the solution, unique identifying problem number (foreign key to Leetcode Problems), solution language, and the author of the solution. They can be joined with a natural join on unique identifying problem number to form a full workable table - with functional dependencies. 

This dataset serves as a diverse representation of human responses to controlled coding assignments. 

The two sheets will be opened using pandas.

In [4]:
problems = pd.read_excel(os.path.join(data_dir, "Leetcode Problems.xlsx"), sheet_name="Sheet1")
problems.rename(columns={'Problem Number': 'ProblemNumber'}, inplace=True)
problems.rename(columns={'Problem Name': 'ProblemName'}, inplace=True)
solutions = pd.read_excel("data/Leetcode Top 150 Solutions.xlsx", sheet_name="Sheet1")
solutions.rename(columns={'Problem Number': 'ProblemNumber'}, inplace=True)

In [5]:
problems.head()

,ProblemName,Link,ProblemNumber,Subject,Difficulty
0,Merge Sorted Array,https://leetcode.com/problems/merge-sorted-arr...,88,Array / String,Easy
1,Remove Element,https://leetcode.com/problems/remove-element/d...,27,Array / String,Easy
2,Remove Duplicates from Sorted Array,https://leetcode.com/problems/remove-duplicate...,26,Array / String,Easy
3,Remove Duplicates from Sorted Array II,https://leetcode.com/problems/remove-duplicate...,80,Array / String,Medium
4,Majority Element,https://leetcode.com/problems/majority-element...,169,Array / String,Easy


In [6]:
solutions.head()

,Solution,Size,Language,ProblemNumber,Author
0,class Solution {\npublic:\n void merge(vect...,247,C++,88,amanrathore48
1,class Solution {\n public void merge(int[] ...,221,Java,88,amanrathore4
2,"class Solution(object):\n def merge(self, n...",140,Python,88,amanrathore4
3,class Solution {\npublic:\n int removeEleme...,295,C++,27,rahulvarma5300
4,class Solution {\n public int removeElement...,291,Java,27,kitt


## Converting the tables to sqlite dbs
Many aggregation functions are much easier to evaluate with sqlite. Using python we'll create a db with 2 tables and use sqlite for some analysis. 

In [7]:
db_exists = False

if not db_exists:
    db_name = 'leetcode.db'

    # Connect to database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()


    cursor.execute('''
        CREATE TABLE IF NOT EXISTS LeetcodeTop150Problems (
            ProblemName TEXT,
            Link TEXT,
            ProblemNumber INTEGER,
            Subject TEXT,
            Difficulty TEXT
        )
    ''')


    problems.to_sql('LeetcodeTop150Problems', conn, if_exists='replace', index=False)


    cursor.execute('''
        CREATE TABLE IF NOT EXISTS LeetcodeTop150Solutions (
            Solution TEXT,
            Size INTEGER,
            Language TEXT,
            ProblemNumber INTEGER,
            Author TEXT
        )
    ''')


    solutions.to_sql('LeetcodeTop150Solutions', conn, if_exists='replace', index=False)


    conn.commit()
    conn.close()

## Summary Statistics from SQL
#### Count # problems for each difficulty </br>
Easy: 117</br>
Medium: 279</br>
Hard: 53</br>
</br>
#### Sanity Checks:</br>
Count # of unique problems = 150</br>
Count # of problems without 3 records in the table = 0</br>
Count # of languages = 3 </br>

#### Authors: </br>
Count unique authors = 222 </br>
Top 3 Most Frequent Authors = deleted_user(s), OldCodingFarmer, StefanPochmann</br>

#### Tokenization: </br>
size_of_all_text = 306,207

In [14]:
adversarial_data = pd.read_csv(os.path.join(data_dir, "llm_generated_solutions.csv"))
adversarial_data.head()

,Solution,Size,Language,ProblemName,model
0,class Solution {\npublic:\n void merge(vect...,247,C++,Merge Sorted Array,BLOOM
1,class Solution {\npublic:\n void merge(vect...,247,C++,Merge Sorted Array,CodeLlama
2,class Solution {\npublic:\n void merge(vect...,247,C++,Merge Sorted Array,Gemma
3,class Solution {\n public void merge(int[] ...,221,Java,Merge Sorted Array,BLOOM
4,class Solution {\n public void merge(int[] ...,221,Java,Merge Sorted Array,CodeLlama
